In [ ]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

# WandB - Install the W&B library
!pip install wandb -q

# Need awscli to download images from AWS (labels come from GCP)
!pip install awscli

print("#### Versions: ####")
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

In [ ]:
import logging
import os
import argparse
from collections import OrderedDict
import pandas as pd
import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.engine import default_argument_parser, default_setup, launch
from detectron2.evaluation import ( ### Changes from original, I don't need all the different evaluators
    COCOEvaluator,
    DatasetEvaluators,
    inference_on_dataset,
    print_csv_format
)
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
# Not sure what these do but they may help to track experiments
from detectron2.utils.events import (
    CommonMetricPrinter,
    EventStorage,
    JSONWriter,
    TensorboardXWriter
)

from detectron2.utils.visualizer import Visualizer

# Setup logger
logger = logging.getLogger("detectron2")

In [ ]:
# Download all images for train & test & validation
!python3 downloadOI.py --classes 'Bed, Cabinetry, Chair, Couch, Lamp, Table' --dataset train, validation, test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
target_classes = ['Chair', 'Cabinetry', 'Couch', 'Bed', 'Table', 'Lamp']
target_classes.sort()
target_classes

['Bed', 'Cabinetry', 'Chair', 'Couch', 'Lamp', 'Table']

In [ ]:
#Setup training, test & validation data paths
valid_path = "/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442"
train_path = "/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-train-20201208-1500perClass-8765"
test_path = "/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254"

valid_annotation_file = "/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/validation-annotations-bbox.csv"
train_annotation_file = "/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/train-annotations-bbox.csv"
test_annotation_file = "/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/test-annotations-bbox.csv"

In [ ]:
def get_image_ids(image_folder=None):
    """
    Explores a folder of images and gets their ID from their file name.
    Returns a list of all image ID's in image_folder.
    E.g. image_folder/608fda8c976e0ac.jpg -> ["608fda8c976e0ac"]
    
    Params
    ------
    image_folder (str): path to folder of images, e.g. "../validation/"
    """
    return [os.path.splitext(img_name)[0] for img_name in os.listdir(image_folder) if img_name.endswith(".jpg")]

In [ ]:
# Make a function which formats a specific annotations csv based on what we're dealing with (relative to absolute pixel values)
def format_annotations(image_folder, annotation_file, target_classes=None):
    """
    TODO - NOTE: This function could (definitely can) be faster.
    TODO - Some ideas: skip the use of pandas entirely and use CSV's
    
    Formats annotation_file based on images contained in image_folder.
    Will get all unique image IDs and make sure annotation_file
    only contains those (the target images).
    Adds meta-data to annotation_file such as class names and categories.
    If target_classes isn't None, the returned annotations will be filtered by this list.
    Note: image_folder and annotation_file should both be validation if working on
    validation set or both be training if working on training set.
    
    Params
    ------
    image_folder (str): path to folder of target images.
    annotation_file (str): path to annotation file of target images.
    target_classes (list), optional: a list of target classes you'd like to filter labels.
    """
    # Get all image ids from target directory
    image_ids = get_image_ids(image_folder)
    
    # Setup annotation file and classnames
    # TODO - improve this, is pandas required? 
    annot_file = pd.read_csv(annotation_file)
    classes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/class-descriptions-boxable.csv",
                          names=["LabelName", "ClassName"])
    
    # Create classname column on annotations which converts label codes to string labels
    annot_file["ClassName"] = annot_file["LabelName"].map(classes.set_index("LabelName")["ClassName"])

    # Sort annot_file by "ClassName" for alphabetical labels (used with target_classes)
    annot_file.sort_values(by=["ClassName"], inplace=True)
    
    # TODO - fix this, Make sure we only get the images we're concerned about
    if target_classes:
        annot_file = annot_file[annot_file["ImageID"].isin(image_ids) & annot_file["ClassName"].isin(target_classes)]
    else:
        annot_file = annot_file[annot_file["ImageID"].isin(image_ids)]
   
    # Add ClassID column, e.g. "Bathtub, Toilet" -> 1, 2
    annot_file["ClassName"] = pd.Categorical(annot_file["ClassName"])
    annot_file["ClassID"] = annot_file["ClassName"].cat.codes
    
    return annot_file

In [ ]:
def rel_to_absolute(bbox, height, width):
    """
    Converts bounding box dimensions from relative to absolute pixel values (Detectron2 style).
    See: https://detectron2.readthedocs.io/modules/structures.html#detectron2.structures.BoxMode
    
    Params
    ------
    bbox (array): relative dimensions of bounding box in format (x0, y0, x1, y1 or Xmin, Ymin, Xmax, Ymax)
    height (int): height of image
    width (int): width of image
    """
    bbox[0] = np.round(np.multiply(bbox[0], width)) # x0
    bbox[1] = np.round(np.multiply(bbox[1], height)) # y0
    bbox[2] = np.round(np.multiply(bbox[2], width)) # x1
    bbox[3] = np.round(np.multiply(bbox[3], height)) # y1
    return [i.astype("object") for i in bbox] # convert all to objects for JSON saving

In [ ]:
# Import CV2 for getting height & width of image
import cv2

# Import Detectron2 BoxMode for bounding boxes style
from detectron2.structures import BoxMode

In [ ]:
import numpy as np
from detectron2.structures import BoxMode
import json
import os
from tqdm import tqdm

def get_image_dicts(image_folder, annotation_file, target_classes=None):
    """
    Create JSON of dectectron2 style labels to be reused later.
    
    TODO -- Maybe create some verbosity here? AKA, what are the outputs?
    TODO -- what if annotations = None? Can we create a call to create an annotations CSV in 1 hit?
    
    Params
    ------
    image_folder (str): target folder containing images
    annotations (DataFrame): DataFrame of image label data
    """
    # dataset_name = "validation" if "valid" in image_folder else "train"

    if "valid" in image_folder:
      dataset_name = "validation"
    elif "train" in image_folder:
      dataset_name = "train"
    elif "test" in image_folder:
      dataset_name = "test"

    print(f"Using {annotation_file} for annotations...")
    # TODO: there should be some kind of asssertions here making sure the image folder and annotation files match
    # E.g. train w/ train and valid w/ valid
    annotations = format_annotations(image_folder=image_folder, 
                                     annotation_file=annotation_file,
                                     target_classes=target_classes)

    print(f"On dataset: {dataset_name}")
    print("Classes we're using:\n {}".format(annotations["ClassName"].value_counts()))

    # Get all unique image ids from target folder
    img_ids = get_image_ids(image_folder)
    print(f"Total number of images: {len(img_ids)}")

    # TODO: move img_data creation out of for loop and only work with subset of img_ids?
    #img_data = annotations[annotations["ImageID"] == img].reset_index() # reset index important for images with multiple objects
    #change to something like "img_data = annotations is in img_ids..."
    
    # Start creating image dictionaries (Detectron2 style labelling)
    img_dicts = []
    for idx, img in tqdm(enumerate(img_ids)):
        record = {}
        
        # Get image metadata
        file_name = image_folder + "/" + img + ".jpg"
        height, width = cv2.imread(file_name).shape[:2]
        img_data = annotations[annotations["ImageID"] == img].reset_index() # reset index important for images
                                                                            # with multiple objects
        # Verbosity for image label troubleshooting
        # print(f"On image: {img}")
        # print(f"Image category: {img_data.ClassID.values}")
        # print(f"Image label: {img_data.ClassName.values}")

        # Update record dictionary
        record["file_name"] = file_name
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        # Create list of image annotations (labels)
        img_annotations = []
        for i in range(len(img_data)): # this is where we loop through examples with multiple objects in an image
            category_id = img_data.loc[i]["ClassID"].astype("object") # JSON (for evalution) can't take int8 (NumPy type) must be native Python type
            # print(f"Image category 2: {category_id}")
            # Get bounding box coordinates in Detectron2 style (x0, y0, x1, y1)
            bbox = np.float32(img_data.loc[i][["XMin", "YMin", "XMax", "YMax"]].values) # needs to be float/int # TODO: change for JSON
            # Convert bbox from relative to absolute pixel dimensions
            bbox = rel_to_absolute(bbox=bbox, height=height, width=width)
            # Setup annot (1 annot = 1 label, there might be more) dictionary
            annot = {
                "bbox": bbox, 
                "bbox_mode": BoxMode.XYXY_ABS, # See: https://detectron2.readthedocs.io/modules/structures.html#detectron2.structures.BoxMode.XYXY_ABS
                "category_id": category_id
            }
            img_annotations.append(annot)
            
        # Update record dictionary with annotations
        record["annotations"] = img_annotations
        
        # Add record dictionary with image annotations to img_dicts list
        img_dicts.append(record)

    # TODO: Change this into it's own function??
    # Save img_dicts to JSON for use later
    json_file = os.path.join(image_folder, dataset_name+"_labels.json")
    print(f"Saving labels to: {json_file}...")
    with open(json_file, "w") as f:
      json.dump(img_dicts, f)

    # return img labels dictionary
    return img_dicts

In [ ]:
# Generate dictionary to store annotation info of validation set
%%time
valid_img_dicts = get_image_dicts(valid_path, valid_annotation_file, target_classes=target_classes)

Using /content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/validation-annotations-bbox.csv for annotations...


7it [00:00, 59.48it/s]

On dataset: validation
Classes we're using:
 Table        316
Chair        275
Cabinetry    150
Bed          109
Lamp          70
Couch         44
Name: ClassName, dtype: int64
Total number of images: 431


431it [00:09, 47.77it/s]

Saving labels to: /content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/validation_labels.json...
CPU times: user 8.15 s, sys: 263 ms, total: 8.41 s
Wall time: 9.52 s


In [ ]:
# Generate dictionary to store annotation info of validation set
%%time
test_img_dicts = get_image_dicts(test_path, test_annotation_file, target_classes=target_classes)

Using /content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/test-annotations-bbox.csv for annotations...


6it [00:00, 50.53it/s]

On dataset: train
Classes we're using:
 Table        891
Chair        672
Cabinetry    426
Bed          324
Lamp         116
Couch        104
Name: ClassName, dtype: int64
Total number of images: 1220


1220it [00:26, 46.26it/s]

Saving labels to: /content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/train_labels.json...
CPU times: user 24 s, sys: 733 ms, total: 24.7 s
Wall time: 27.9 s


In [ ]:
# Check if there are duplicated images in annotation dicts (validation: 431 images; test: 1220 images)
print(len(valid_img_dicts))
print(len(test_img_dicts))

442
1254


In [ ]:
# Check classes of images in validation set (ignore images without class assigned)
unique_cats_valid = []
for idx, dicty in enumerate(valid_img_dicts):
  try:
    unique_cats_valid.append(dicty["annotations"][0]["category_id"])
  except:
    pass
print(f"Unique categories in valid_img_dicts: {set(unique_cats_valid)}")


Unique categories in valid_img_dicts: {0, 1, 2, 3, 4, 5}


In [ ]:
# Check and Remove images which are not in annotation csv
imgID_problemed_valid = []
unique_cats_valid = []
for idx, dicty in enumerate(valid_img_dicts):
  try:
    unique_cats_valid.append(dicty["annotations"][0]["category_id"])
  except:
    imgID_problemed_valid.append(dicty["file_name"])
    pass
imgID_problemed_valid

['/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/03646bb2f288eaae.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/0fb3a760f909c8c9.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/20af1d21449acdce.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/2110e5d8ca7e5a99.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/36fb4da1df30f769.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201208-100perClass-442/5a44d26db9ea0aa8.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-validation-20201

In [ ]:
# Success: 6 target classes available
unique_cats_valid = []
for dicty in valid_img_dicts:
  unique_cats_valid.append(dicty["annotations"][0]["category_id"])
print(f"Unique categories in valid_img_dicts: {set(unique_cats_valid)}")


Unique categories in valid_img_dicts: {0, 1, 2, 3, 4, 5}


In [ ]:
# Check classes of images in test set (ignore images without class assigned)
unique_cats_test = []
for idx, dicty in enumerate(test_img_dicts):
  try:
    unique_cats_test.append(dicty["annotations"][0]["category_id"])
  except:
    pass
print(f"Unique categories in test_img_dicts: {set(unique_cats_test)}")


Unique categories in test_img_dicts: {0, 1, 2, 3, 4, 5}


In [ ]:
# Check and Remove images which are not in annotation csv
imgID_problemed_test = []
unique_cats_test = []
for idx, dicty in enumerate(test_img_dicts):
  try:
    unique_cats_test.append(dicty["annotations"][0]["category_id"])
  except:
    imgID_problemed_test.append(dicty["file_name"])
    pass
imgID_problemed_test

['/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/2aa83f4d9dbfdc54.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/360d48f9e44b9609.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/58660f6531f19087.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/58bc111b3ba43cd9.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/5b5137015349bbc2.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/5fcede006318c905.jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-test-20201208-300perClass-1254/6c34953ed63c6d6

In [ ]:
# Success: 6 target classes available
unique_cats_test = []
for dicty in test_img_dicts:
  unique_cats_test.append(dicty["annotations"][0]["category_id"])
print(f"Unique categories in test_img_dicts: {set(unique_cats_test)}")


Unique categories in test_img_dicts: {0, 1, 2, 3, 4, 5}


In [ ]:
# Check again if there are duplicated images in annotation dicts (validation: 431 images; test: 1220 images)
print(len(valid_img_dicts))
print(len(test_img_dicts))

431
1220


In [ ]:
# Generate dictionary to store annotation info of training set
%%time
train_img_dicts = get_image_dicts(train_path, train_annotation_file, target_classes=target_classes)

Using /content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/train-annotations-bbox.csv for annotations...
On dataset: train
Classes we're using:
 Chair        12189
Table         5905
Cabinetry     4664
Lamp          3527
Couch         2024
Bed           1902
Name: ClassName, dtype: int64


0it [00:00, ?it/s]

Total number of images: 8765


8765it [47:36,  3.07it/s]


Saving labels to: /content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-train-20201208-1500perClass-8765/train_labels.json...
CPU times: user 4min 15s, sys: 13.8 s, total: 4min 28s
Wall time: 48min 22s


In [ ]:
# Check classes of images in training set
unique_cats_train = []
for dicty in train_img_dicts:
  unique_cats_train.append(dicty["annotations"][0]["category_id"])
print(f"Unique categories in train_img_dicts: {set(unique_cats_train)}")

IndexError: ignored

In [ ]:
# Check classes of images in training set (ignore images without class assigned)
unique_cats_train = []
for idx, dicty in enumerate(train_img_dicts):
  try:
    unique_cats_train.append(dicty["annotations"][0]["category_id"])
  except:
    pass
print(f"Unique categories in train_img_dicts: {set(unique_cats_train)}")


Unique categories in train_img_dicts: {0, 1, 2, 3, 4, 5}


In [ ]:
len(train_img_dicts)

8765

In [ ]:
# Check and Remove images which are not in annotation csv (duplicated and filenames added with (1))
imgID_problemed_train = []
unique_cats_train = []
for idx, dicty in enumerate(train_img_dicts):
  try:
    unique_cats_train.append(dicty["annotations"][0]["category_id"])
  except:
    imgID_problemed_train.append(dicty["file_name"])
    pass
imgID_problemed_train

['/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-train-20201208-1500perClass-8765/f7639ed2a25d18b4 (1).jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-train-20201208-1500perClass-8765/e95eb37f50a0a308 (1).jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-train-20201208-1500perClass-8765/6a888569fdce9b16 (1).jpg',
 '/content/drive/MyDrive/Colab Notebooks/airbnb-amenity-detection/training_data/all-final-train-20201208-1500perClass-8765/6cf9c2060724f007 (1).jpg']

In [ ]:
# Check number of images in 3 sets after removal of problematic images
print(len(train_img_dicts))
print(len(valid_img_dicts))
print(len(test_img_dicts))

8761
431
1220


In [ ]:
def load_json_labels(image_folder):
    """
    Returns Detectron2 style labels of images in image_folder based on JSON label file in image_folder.
    
    TODO -- Maybe create some verbosity here? AKA, what are the outputs?
    TODO -- what if annotations = None? Can we create a call to create an annotations CSV in 1 hit?
    
    Params
    ------
    image_folder (str): target folder containing images
    """
    # Get absolute path of JSON label file
    for file in os.listdir(image_folder):
      if file.endswith(".json"):
        json_file = os.path.join(image_folder, file)

    # TODO: Fix this assertion
    assert json_file, "No .json label file found, please make one with annots_to_json()"

    with open(json_file, "r") as f:
      img_dicts = json.load(f)

    # Convert bbox_mode to Enum of BoxMode.XYXY_ABS (doesn't work loading normal from JSON)
    for img_dict in img_dicts:
      for annot in img_dict["annotations"]:
        annot["bbox_mode"] = BoxMode.XYXY_ABS

    return img_dicts

In [ ]:
# Create json files to store annotation info (which is required to run Detectron2 model)
%%time
train_img_dicts = load_json_labels(train_path)
valid_img_dicts = load_json_labels(valid_path)
test_img_dicts = load_json_labels(test_path)

CPU times: user 162 ms, sys: 9 ms, total: 171 ms
Wall time: 305 ms
